# Simple Test for loading and training on 12 x 256 data


In [1]:
import numpy as np

import tensorflow as tf

import h5py

import keras
import keras.backend.tensorflow_backend as k
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.utils import plot_model

import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import PanTool, WheelZoomTool, ResetTool, SaveTool, BoxZoomTool, TapTool, ZoomInTool, ZoomOutTool
from bokeh.models import HoverTool, BoxSelectTool, Rect, CustomJS, ColumnDataSource
from bokeh.io import output_notebook
from bokeh import palettes

#from IPython.html import widgets
import ipywidgets as widgets
import json

print("Keras Version ", keras.__version__)
print("Tensorflow Version ", tf.__version__)
print("Numpy Version ", np.__version__)
print("Bokeh Version ", bokeh.__version__)
print("H5py Version ", h5py.__version__)
print("JSON Version ", json.__version__)
print("Ipywidgets Version ", widgets.__version__)
np.random.seed(1337)  # for reproducibility

C:\Users\djbau\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Keras Version  2.2.2
Tensorflow Version  1.9.0
Numpy Version  1.14.3
Bokeh Version  0.13.0
H5py Version  2.7.1
JSON Version  2.0.9
Ipywidgets Version  7.2.1


This tensorflow wizardry seems to help with GPU memory allocation issues. The notebook will only allocate as much memory as needed, up to a chosen fraction of the available memory. 

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# Only allow a fraction of the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 1.0
session = tf.Session(config=config)
k.set_session(session)

Network parameters:

Preprocess the data so that all values are in the [0,1] interval and encode the labels

Build the CNN. 

In [3]:
model = models.load_model('BatchNormIsoFix.h5')

Show a summary of the model parameters. 

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
a (LocallyConnected2D)       (None, 22, 254, 32)       1788160   
_________________________________________________________________
b (BatchNormalization)       (None, 22, 254, 32)       128       
_________________________________________________________________
c (MaxPooling2D)             (None, 11, 127, 32)       0         
_________________________________________________________________
d (LocallyConnected2D)       (None, 9, 125, 24)        7803000   
_________________________________________________________________
e (BatchNormalization)       (None, 9, 125, 24)        96        
_________________________________________________________________
f (Flatten)                  (None, 27000)             0         
_________________________________________________________________
g (Dense)                    (None, 24)                648024    
__________

And now run the model and evaluate on a test set (drawn from the same HDF5 file). 

In [5]:
h = h5py.File('BatchNormIsoFix.h5', 'r')
sample_ds = h['RealPower/Sample']
sample_data = sample_ds[...]
min_data = np.min(sample_data)
max_data = np.max(sample_data)
# In future, this will subtract the mean and divide by the std. dev.
sample_data = (sample_data - min_data) / (max_data - min_data)
#sample_data.resize(256, len(sample_data), 1)
sample_data = sample_data.reshape(sample_data.shape[0], sample_data.shape[1], 1)
sample_data.shape

def make_samples_from_data(surf, root_name="surface", required_width=256, sample_height=24, positive=0):
    surface_shape = surf.shape
    pieces = None
    if surface_shape[1] != 256:
        return pieces

    for i in range(surface_shape[0] - sample_height):
        a_slice = surf[i: i+sample_height]
        if pieces is None:
            pieces = np.zeros((surface_shape[0] - sample_height, sample_height, 256, 1))
        pieces[i] = a_slice[:]
    return pieces
all_slices = make_samples_from_data(sample_data)

In [6]:
# Stash the predictions in a hidden HTML div, in JSON format for use in the JavaScript callback
all_slices.shape
all_predictions = model.predict(all_slices)
# Just so the notebook does something interesting, stick in some zeros at 100 - 110
all_actual = np.ones_like(all_predictions)
all_actual[100:110] = 0
session.close()
sample_stats = {'actual': all_actual.flatten().tolist(), 'predictions': all_predictions.flatten().tolist()}
predictions_json = json.dumps(sample_stats)
widgets.HTML(value = '''<div  class='json_data hidden'> ''' + predictions_json + ' </div> ',
            placeholder='JSON Data',
            description='JSON Data')


HTML(value='<div  class=\'json_data hidden\'> {"actual": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.…

The next cell contains Javascript code to allow the Javascript widget callback to execute Python code to 
update the slider widget values when the user moves the selection box over the bokeh plot. For now, this will mostly not show anything because the prediction is always 1 for this example, and the actual value is also always one. I've put some fake zeros in the range of 100 - 120, so when you drag over those you'll see the status change.

In [7]:
%%javascript
window.executePython = function(python) {
    return new Promise((resolve, reject) => {
        var callbacks = {
            iopub: {
                output: (data) => resolve(data.content.text.trim())
            }
        };
        Jupyter.notebook.kernel.execute(`${python}`, callbacks);    
    });
}

// Javascript code to enable Javascript to call Python to update widget state


<IPython.core.display.Javascript object>

In [9]:
source = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[]))

callback = CustomJS(args=dict(source=source), code="""
        // get data source from Callback args
        var data = source.data;
        var jsString = $('div.json_data').text();
        var jsData = $.parseJSON(jsString);
    
        //alert(predictions.actual);
        /// get BoxSelectTool dimensions from cb_data parameter of Callback
        var geometry = cb_data['geometry'];

        /// calculate Rect attributes
        var width = 256; //geometry['x1'] - geometry['x0'];
        var height = 24; //geometry['y1'] - geometry['y0'];
        var x = 0; 
        var y = parseInt(+geometry['y0'] +""); 

        var a = jsData.actual[y];
        var p = jsData.predictions[y];
        
        /// update data source with new Rect attributes
        data['x'] = []
        data['y'] = []
        data['x'].push(0);
        data['y'].push(y);
        //data['width'].push(width);
        //data['height'].push(height);

        function Log_out(r)
        { console.log(r); };

        var code = 
        'prediction_widget.value = ' + p + '; actual_widget.value = ' + a;
    
        window.executePython( code )
            .then(result => Log_out(result)); // Log out
    
        // emit update of data source    
        source.change.emit();
    """)

box_select = BoxSelectTool(callback=callback)
output_notebook()

rect = Rect(x='x', y='y', width=256, height=24, fill_alpha=0.9, fill_color='#009933')

p = figure(
    title='Doppler Sample',
    plot_width=512,
    plot_height = sample_data.shape[0] * 2,
    x_range=[0,256],
    y_range=[0, sample_data.shape[0]],
    tooltips=[("x",'$x'), ("y", "$y"), ("value", "@image")],
    tools=[BoxSelectTool(dimensions='height', callback=callback),
           HoverTool(),
           ZoomInTool(),
           ZoomOutTool(),
           ResetTool()],
)
p.image(image=[sample_data], x=0, y=0, dw=256, dh=sample_data.shape[0], palette='Plasma256')
p.add_glyph(source, rect, selection_glyph=rect, nonselection_glyph=rect)
xs = range(256)
ys = range(sample_data.shape[0])

p.grid.grid_line_color = None
show(p)


prediction_widget = widgets.FloatSlider(
    description='Prediction',
    value=1,
    min=0,
    max=1.0,
    step=0.1,
    disabled=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)
prediction_widget.add_class('prediction')
actual_widget  = widgets.FloatSlider(
    description='Actual',
    value=1,
    min=0,
    max=1.0,
    step=0.1,
    disabled=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)
actual_widget.add_class('actual')
def f(p, a):
    if abs(p - a) < 0.2:
        cond = 'Good Prediction'
    else:
        cond = 'Bad Prediction'
    print('Prediction: {} Actual: {} {}'.format(p, a, cond))

out = widgets.interactive_output(f, {'p': prediction_widget, 'a': actual_widget})

widgets.HBox([widgets.VBox([prediction_widget, actual_widget]), out])

Loading BokehJS ...